          Open, High, Low, Close, Volume, Dividends, Stock Splits                        
1. 09:15
2. 10:15
3. 11:15
4. 12:15
5. 13:15
6. 14:15
7. 15:15

In [ ]:
!pip install transformers sentence-transformers sacremoses
!pip install yfinance --upgrade --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 68.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=e1d1668bfac9f46662c50d2945988cf08153794f68d9589f391451aba7277d2a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe85104

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import yfinance as yf
import transformers
import torch
from torch import nn
from sentence_transformers import SentenceTransformer

In [ ]:
PATH = r'/content/drive/MyDrive/training_check_point.pt'
checkpoint = torch.load(PATH)

<All keys matched successfully>

In [ ]:
# get list of symbols available

symbols = list()
list_loc = r"/content/drive/MyDrive/STOCKS.txt"
with open(list_loc, 'r') as f:
  for line in f.readlines():
    symbols.append(line.strip())

print(len(symbols), symbols[0], symbols[len(symbols)-1])

18 INFY COALINDIA


In [ ]:
# Basis information by ticker name

def get_hist(ticker_name):
  """Divide the input dictinary into multiple seq with specified length.

  Args:
    ticker_name: A SYMBOL.

  Returns:
    info: A dict with basic information about  the given symbol.
    hist: historical market data of period 1 day and interval of 1 min
  """
  # get basic data
  ticker = yf.Ticker(ticker_name)

  # get historical market data
  hist = ticker.history(period="1d", interval="1h")

  return hist

In [ ]:
hist = get_hist('INFY.NS')
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-10-23 09:15:00+05:30,1426.000000,1432.349976,1424.000000,1426.750000,0,0.0,0.0
2023-10-23 10:15:00+05:30,1426.099976,1426.550049,1418.449951,1419.750000,376589,0.0,0.0
2023-10-23 11:15:00+05:30,1419.750000,1420.099976,1416.550049,1418.000000,389961,0.0,0.0
2023-10-23 12:15:00+05:30,1418.349976,1419.900024,1415.699951,1418.900024,206589,0.0,0.0
2023-10-23 13:15:00+05:30,1418.900024,1419.500000,1413.550049,1416.750000,248080,0.0,0.0
2023-10-23 14:15:00+05:30,1416.199951,1417.349976,1405.250000,1406.699951,532930,0.0,0.0
2023-10-23 15:15:00+05:30,1406.949951,1410.050049,1406.550049,1408.650024,354806,0.0,0.0


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-10-23 09:15:00+05:30,1426.000000,1432.349976,1424.000000,1426.750000,0,0.0,0.0
2023-10-23 10:15:00+05:30,1426.099976,1426.550049,1418.449951,1419.750000,376589,0.0,0.0
2023-10-23 11:15:00+05:30,1419.750000,1420.099976,1416.550049,1418.000000,389961,0.0,0.0
2023-10-23 12:15:00+05:30,1418.349976,1419.900024,1415.699951,1418.900024,206589,0.0,0.0
2023-10-23 13:15:00+05:30,1418.900024,1419.500000,1413.550049,1416.750000,248080,0.0,0.0
2023-10-23 14:15:00+05:30,1416.199951,1417.349976,1405.250000,1406.699951,532930,0.0,0.0
2023-10-23 15:15:00+05:30,1406.949951,1410.050049,1406.550049,1408.650024,354806,0.0,0.0


In [ ]:
def hist_into_lebel(hist):
  """Divide the input news list into multiple seq with specified length.

  Args:
    hist: A dataframe.

  Returns:
    Hourly prediction
    List with time stamp
  """
  seq_li = list()
  time = list()
  no_of_row = hist.shape[0]
  for i in range(no_of_row):
    #print(hist.iloc[i].to_string().strip() + " at " + str(hist.iloc[i].name))
    time.append(hist.iloc[i].name)
    seq_li.append([hist.iloc[i]['Open'], hist.iloc[i]['High'], hist.iloc[i]['Low'], hist.iloc[i]['Close'], hist.iloc[i]['Volume'], hist.iloc[i]['Dividends'], hist.iloc[i]['Stock Splits']])
  return torch.tensor(seq_li, dtype=torch.float32).flatten(start_dim=0), time



In [ ]:
hist1, time = hist_into_lebel(get_hist('INFY.NS'))
print(hist1)
print(time)

tensor([  1426.0000,   1432.3500,   1424.0000,   1426.7500,      0.0000,
             0.0000,      0.0000,   1426.1000,   1426.5500,   1418.4500,
          1419.7500, 376589.0000,      0.0000,      0.0000,   1419.7500,
          1420.1000,   1416.5500,   1418.0000, 389961.0000,      0.0000,
             0.0000,   1418.3500,   1419.9000,   1415.7000,   1418.9000,
        206589.0000,      0.0000,      0.0000,   1418.9000,   1419.5000,
          1413.5500,   1416.7500, 248080.0000,      0.0000,      0.0000,
          1416.2000,   1417.3500,   1405.2500,   1406.7000, 532930.0000,
             0.0000,      0.0000,   1406.9500,   1410.0500,   1406.5500,
          1408.6500, 354806.0000,      0.0000,      0.0000])
[Timestamp('2023-10-23 09:15:00+0530', tz='Asia/Kolkata'), Timestamp('2023-10-23 10:15:00+0530', tz='Asia/Kolkata'), Timestamp('2023-10-23 11:15:00+0530', tz='Asia/Kolkata'), Timestamp('2023-10-23 12:15:00+0530', tz='Asia/Kolkata'), Timestamp('2023-10-23 13:15:00+0530', tz='Asia/Ko

In [ ]:
hist1.shape

torch.Size([49])

In [ ]:
import transformers
import torch
from torch import nn
from sentence_transformers import SentenceTransformer
class BERTRegression(nn.Module):
    def __init__ (self):
        super(BERTRegression, self).__init__()
        #self.bert = transformers.BertModel.from_pretrained('bert-base-cased') # distilbert-base-uncased
        self.flat = nn.Flatten(start_dim=0)
        self.embeding = SentenceTransformer('paraphrase-MiniLM-L6-v2')
        self.bert = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-cased')
        self.reg_layer = nn.Sequential(
            nn.Linear(384*768, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, 500),
            nn.ReLU(inplace=True),
            nn.Linear(500, 49),
            nn.ReLU(inplace=True),
        )

    def forward(self, input):
        ids = torch.LongTensor(self.embeding.encode(input)).unsqueeze(0)
        print(ids.shape)
        output = self.bert(ids)
        output = self.flat(output[0])
        output = self.reg_layer(output)

        return output

/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/huggingface/pytorch-transformers/zipball/main" to /root/.cache/torch/hub/main.zip


In [ ]:
# LOSS FUNCGION AND OPTIMIZER
model = BERTRegression()
model.load_state_dict(checkpoint['model_state_dict'])
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
import pickle
pred_save_loc = r"/content/drive/MyDrive/PREDICTION/"
def load(path: str):
  pred = 0
  with open(path, "rb") as f:
    pred = pickle.load(f)
  return pred
for symbol in symbols:
    pred = load(pred_save_loc + symbol +'.pkl')
    print(pred)

tensor([0.0000, 0.8109, 0.1118, 0.0000, 0.2259, 0.0000, 0.0070, 0.2987, 0.0000,
        0.0000, 0.1071, 0.0000, 0.0515, 0.0000, 0.0000, 0.5069, 0.1734, 0.4316,
        0.5053, 0.2485, 0.3255, 0.0000, 0.1677, 0.0000, 0.1105, 0.6774, 0.5894,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5636,
        0.4822, 0.0596, 0.0000, 0.0000, 0.0000, 0.0000, 0.3841, 0.0319, 0.0000,
        0.2309, 0.0000, 0.2506, 0.0456], requires_grad=True)
tensor([0.0000, 0.8109, 0.1118, 0.0000, 0.2259, 0.0000, 0.0070, 0.2987, 0.0000,
        0.0000, 0.1071, 0.0000, 0.0515, 0.0000, 0.0000, 0.5069, 0.1734, 0.4316,
        0.5053, 0.2485, 0.3255, 0.0000, 0.1677, 0.0000, 0.1105, 0.6774, 0.5894,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5636,
        0.4822, 0.0596, 0.0000, 0.0000, 0.0000, 0.0000, 0.3841, 0.0319, 0.0000,
        0.2309, 0.0000, 0.2506, 0.0456], requires_grad=True)
tensor([0.0000, 0.8109, 0.1118, 0.0000, 0.2259, 0.0000, 0.0070, 0.2987, 0.0000

In [ ]:
# Load Prediction

EPOCH = 100
model.train()
for i in range(EPOCH):
    print('EPOCH : ', i+1, 'out of ', EPOCH)
    for symbol in symbols:
        pred = load(pred_save_loc + symbol +'.pkl')
        y, _ = hist_into_lebel(get_hist(symbol+'.NS'))
        loss = loss_fn(pred, y)
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loss, current = loss.item(), symbol
        print(f"Train loss: {loss:>7f}  ["+symbol+"]")

EPOCH :  1 out of  100
INFY torch.Size([49]) torch.Size([49])
Train loss: 16491200512.000000  [INFY]
HDFCBANK torch.Size([49]) torch.Size([49])
Train loss: 378584825856.000000  [HDFCBANK]
RELIANCE torch.Size([49]) torch.Size([49])
Train loss: 42531323904.000000  [RELIANCE]
ITC torch.Size([49]) torch.Size([49])
Train loss: 106963607552.000000  [ITC]
TATAMOTORS torch.Size([49]) torch.Size([49])
Train loss: 166847889408.000000  [TATAMOTORS]
MARUTI torch.Size([49]) torch.Size([49])
Train loss: 204002704.000000  [MARUTI]
TATASTEEL torch.Size([49]) torch.Size([49])
Train loss: 2233108791296.000000  [TATASTEEL]
TECHM torch.Size([49]) torch.Size([49])
Train loss: 3382657280.000000  [TECHM]
ICICIBANK torch.Size([49]) torch.Size([49])
Train loss: 364293783552.000000  [ICICIBANK]
BAJAJ-AUTO torch.Size([49]) torch.Size([49])
Train loss: 401671904.000000  [BAJAJ-AUTO]
WIPRO torch.Size([49]) torch.Size([49])
Train loss: 51254534144.000000  [WIPRO]
SBIN torch.Size([49]) torch.Size([49])
Train loss: 3

In [ ]:
PATH = r'/content/drive/MyDrive/training_check_point.pt'
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, PATH)